# Let’s use a simple example data to demonstrate how XGBoost algorithm works


### A) on a regression problem.

Let’s use a simple example data to demonstrate how XGBoost algorithm works on a regression problem.


#### Step 1. Set an initial prediction
Let’s set the initial prediction (F0(x)) to be 3. In other words, regardless of the value of X, the predicted Y will be 3 - mean value.


In [2]:
x = [5,8,15,20,-9]
y = [1,2,8,4,0]
pred_0 = sum(y) / len(y)
pred_0

3.0

### Step 2. Calculate the residuals
In this step we calculate the difference between the actual value (Y) and the predicted value (F0(x)).


In [3]:
res_0 = []
for i in range(len(y)):
    res_0.append(y[i] - pred_0)
res_0

[-2.0, -1.0, 5.0, 1.0, -3.0]

### Step 3. Train model on the residuals

Here, we build one or more tree-based models that are trained on [X, res].

The following shows how a tree model is constructed.

#### A) Generate split candidates ####

In this step we find one or more split candidates. The formula is with N rows we would have N - 1 split candidates.

With our dataset example, here’s the result.

The threshold will be used to create two child nodes. 

Here’s the complete parent-child nodes for each candidate.

===== Candidate Split 1 =====

Parent node -> X < -2
Left child node, res_0 -> [-3]
Right child node, res_0 -> [-2, -1, 5, 1]

===== Candidate Split 2 =====

Parent node -> X < 6.5
Left child node -> [-3, -2]
Right child node -> [-1, 5, 1]

===== Candidate Split 3 =====

Parent node -> X < 11.5
Left child node -> [-3, -2, -1]
Right child node -> [5, 1]

===== Candidate Split 4 =====

Parent node -> X < 22.5
Left child node -> [-3, -2, -1, 5]
Right child node -> [1]


#### B) Select the split
From step (A) we generate one or more split candidates. The question is how to select the best one?

In XGBoost, we select the best split by leveraging what are called as Similarity Score and Gain. Gain is calculated for the parent node, while Similarity Score is calculated for the child nodes. We select a candidate with the maximum Gain.

Here are the formulas.

Gain = Left_similarity_score + Right_similarity_score - Root_similarity_score

Similarity_score = SUM(residual_i)**2 / {SUM(prev_proba_i x (1 - prev_proba_i)) + lambda}

But in this case we will consider all possible trees with depth = 2

we have 
lamda = 1
gamma = 5




===== Candidate Split 1 =====

Parent node -> X < -2
Left child node, res_0 -> [-3]
Right child node, res_0 -> [-2, -1, 5, 1]

SS_root = (-3 -2 - 1 + 5 + 1)**2 / (5 + 1) = 0
SS_left = (-3)**2 / (1 + 1)= 9/2 = 4.5
SS_right = (-2 - 1 + 5 + 1)**2 / (4 + 1)= 9/5 = 1.8
Gain = 4.5 + 1.8 - 0 - 5 = 0.3 > 0 => SPLITTING

    ___SPLIT1.1___
    
    left - nothing to split - 1 element
    right:
        ___SPLIT1.11___
        
        Parent node -> X < 6.5
        Left child node -> [-2]
        Right child node -> [-1, 5, 1] 
        
        SS_root = (-2 - 1 + 5 + 1)**2 / (4 + 1)= 9/5 = 1.8
        SS_left = (-2)**2 / (1 + 1)= 4/2 = 2
        SS_right = (- 1 + 5 + 1)**2 / (3 + 1)= 25/4 = 6.25
        Gain = 2 + 6.25 - 1.8 - 5 = 1.45 > 0 => SPLITTING
        
        ___SPLIT1.12___

        Parent node -> X < 11.5
        Left child node -> [-2, -1]
        Right child node -> [5, 1]
        
        SS_root = (-2 - 1 + 5 + 1)**2 / (4 + 1)= 9/5 = 1.8
        SS_left = (-2 - 1)**2 / (2 + 1)= 9/3 = 3
        SS_right = (5 + 1)**2 / (2 + 1)= 36/3 = 12
        Gain = 3 + 12 - 1.8 - 5 = 8.2 > 0 => SPLITTING
        
        ___SPLIT1.13___

        Parent node -> X < 22.5
        Left child node -> [-2, -1, 5]
        Right child node -> [1]
        
        SS_root = (-2 - 1 + 5 + 1)**2 / (4 + 1)= 9/5 = 1.8
        SS_left = (-2 - 1 + 5)**2 / (2 + 1)= 4/3 = 1.3
        SS_right = ( 1)**2 / (1 + 1)= 1/2 = 0.5
        Gain = 1.3 + 0.5 - 1.8 - 5 = -5 < 0 => NO SPLITTING        
        

===== Candidate Split 2 =====

Parent node -> X < 6.5
Left child node -> [-3, -2]
Right child node -> [-1, 5, 1]

SS_root = (-3 -2 - 1 + 5 + 1)**2 / (5 + 1) = 0
SS_left = (-3 - 2)**2 / (2 + 1)= 25/3 = 8.3
SS_right = (- 1 + 5 + 1)**2 / (3 + 1)= 25/4 = 6.25
Gain = 8.3 + 6.25 - 0 - 5 = 9.58 > 0 => SPLITTING

    ___SPLIT2.1___
    
    left:
        ___SPLIT2.11___
        
        Parent node -> X < -2
        Left child node, res_0 -> [-3]
        Right child node, res_0 -> [-2]
        
        SS_root = (-3 - 2)**2 / (2 + 1)= 25/3 = 8.3
        SS_left = (-3)**2 / (1 + 1)= 9/2 = 4.5
        SS_right = (-2)**2 / (1 + 1)= 24/2 = 12
        Gain = 4.5 + 12 - 8.3 - 5 = 3.2 > 0 => SPLITTING
     

    right:
        ___SPLIT2.12___

        Parent node -> X < 11.5
        Left child node -> [-1]
        Right child node -> [5, 1]
        
        SS_root = (- 1 + 5 + 1)**2 / (3 + 1)= 25/4 = 6.25
        SS_left = (- 1)**2 / (1 + 1)= 1/2 = 0.5
        SS_right = (5 + 1)**2 / (2 + 1)= 36/3 = 12
        Gain = 0.5 + 12 - 6.25 - 5 = 1.25 > 0 => SPLITTING
        
        ___SPLIT1.13___

        Parent node -> X < 22.5
        Left child node -> [-1, 5]
        Right child node -> [1]
        
        SS_root = (- 1 + 5 + 1)**2 / (3 + 1)= 25/4 = 6.25
        SS_left = (- 1 + 5)**2 / (2 + 1)= 16/3 = 5.3
        SS_right = ( 1)**2 / (1 + 1)= 1/2 = 0.5
        Gain = 5.3 + 0.5 - 6.25 - 5 = -5.45 < 0 => NO SPLITTING 

===== Candidate Split 3 =====

Parent node -> X < 11.5
Left child node -> [-3, -2, -1]
Right child node -> [5, 1]

SS_root = (-3 -2 - 1 + 5 + 1)**2 / (5 + 1) = 0
SS_left = (-3 - 2 - 1)**2 / (3 + 1)= 36/4 = 9
SS_right = (5 + 1)**2 / (2 + 1)= 36/3 = 12
Gain = 9 + 12 - 0 - 5 = 16 > 0 => SPLITTING

    ___SPLIT3.1___
    
    left:
        ___SPLIT3.11___
        
        Parent node -> X < -2
        Left child node, res_0 -> [-3]
        Right child node, res_0 -> [-2, -1]
        
        SS_root = (-3 - 2 - 1)**2 / (3 + 1)= 36/4 = 9
        SS_left = (-3)**2 / (1 + 1)= 9/2 = 4.5
        SS_right = (-2 - 1)**2 / (2 + 1)= 9/3 = 3
        Gain = 4.5 + 3 - 9 - 5 = -5.5 <0 => NO SPLITTING
        
        ___SPLIT3.12___
        
        Parent node -> X < 6.5
        Left child node, res_0 -> [-3, -2]
        Right child node, res_0 -> [-1]
        
        SS_root = (-3 - 2 - 1)**2 / (3 + 1)= 36/4 = 9
        SS_left = (-3 - 2)**2 / (2 + 1)= 25/3 = 8.3
        SS_right = (- 1)**2 / (1 + 1)= 1/2 = 0.5
        Gain = 8.3 + 0.5 - 9 - 5 = -5.2 <0 => NO SPLITTING
        

    right:
        ___SPLIT3.13___

        Parent node -> X < 22.5
        Left child node -> [5]
        Right child node -> [1]
        
        SS_root = (5 + 1)**2 / (2 + 1)= 36/3 = 12
        SS_left = (5)**2 / (1 + 1)= 25/2 = 12.5
        SS_right = (1)**2 / (1 + 1)= 1/2 = 0.5
        Gain = 12.5 + 0.5 - 12 - 5 = -6 < 0 => NO SPLITTING 
    
        

===== Candidate Split 4 =====

Parent node -> X < 22.5
Left child node -> [-3, -2, -1, 5]
Right child node -> [1]

SS_root = (-3 -2 - 1 + 5 + 1)**2 / (5 + 1) = 0
SS_left = (-3 - 2 - 1 + 5)**2 / (4 + 1)= 1/5 = 0.20
SS_right = (1)**2 / (1 + 1)= 1/2 = 0.5
Gain = 0.20 + 0.5 - 0 - 5 = -4.3 < 0 => NO SPLITTING

Let's take one of the possible trees and take a look on the model
    


===== Candidate Split 2 =====

Parent node -> X < 6.5
Left child node -> [-3, -2]
Right child node -> [-1, 5, 1]

    ___SPLIT2.1___
    
    left:
        ___SPLIT2.11___       
        Parent node -> X < -2
        Left child node, res_0 -> [-3]
        Right child node, res_0 -> [-2]

    right:
        ___SPLIT2.12___
        Parent node -> X < 11.5
        Left child node -> [-1]
        Right child node -> [5, 1]


        


#### c) Calculate Output Values
Output Values is calculated for all leaves of the decision tree.

The formula is given by the following:

In [ ]:
Output Value = SUM(residual_i) / [SUM(n + lambda]


===== Candidate Split 2 =====

Parent node -> X < 6.5
Left child node -> [-3, -2]
Output  = (-3 - 2) / (2 + 1)= -5/3 = -1.6

Right child node -> [-1, 5, 1]
Output = (-1 + 5 -1) / (3 + 1)= 5/3 = 1.6

    ___SPLIT2.1___
    
    left:
        ___SPLIT2.11___
        
        Parent node -> X < -2
        Left child node, res_0 -> [-3]
        Output = (-3) / (1 + 1)= -3/2 = -1.5
        
        Right child node, res_0 -> [-2]
        Output = (-2) / (1 + 1)= -2/2 = -1
      
     
    right:
        ___SPLIT2.12___

        Parent node -> X < 11.5
        Left child node -> [-1]
        Output = (-1) / (1 + 1)= -1/2 = -0.5
        
        Right child node -> [5, 1]
        Output = (5 + 1) / (2 + 1)= 6/3 = 2

        

#### d) Predict the new values using our chosen DT

pred_1 = pred_0 + learning_rate * (DT_0(x))
Where: 
        pred_0 = 3 
        learning_rate = 0.1

### B) on a classification problem.

Let’s use a simple example data to demonstrate how XGBoost algorithm works on a regression problem.

In [ ]:
X	|	Y
==========
-7	|	0
5	|	0
6	|	1
8	|	1
9 	|   1 